# 0. 준비 과정

In [ ]:
# 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 데이터셋 압축 해제
!unzip /content/drive/MyDrive/hole_dataset.ZIP -d /content/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/synthetic_data/train/segmentations/file_000006344.png  
  inflating: /content/synthetic_data/train/segmentations/file_000006345.png  
  inflating: /content/synthetic_data/train/segmentations/file_000006346.png  
  inflating: /content/synthetic_data/train/segmentations/file_000006347.png  
  inflating: /content/synthetic_data/train/segmentations/file_000006348.png  
  inflating: /content/synthetic_data/train/segmentations/file_000006349.png  
  inflating: /content/synthetic_data/train/segmentations/file_000006350.png  
  inflating: /content/synthetic_data/train/segmentations/file_000006351.png  
  inflating: /content/synthetic_data/train/segmentations/file_000006352.png  
  inflating: /content/synthetic_data/train/segmentations/file_000006353.png  
  inflating: /content/synthetic_data/train/segmentations/file_000006354.png  
  inflating: /content/synthetic_data/train/segmentations/file_000006355.png  
  inflating: /content/synthe

# 1-1단계: 데이터 구조 파악 및 전처리

- coco_segmentations.json 파일을 분석해서 각 inspection_id별로 어떤 이미지들이 있는지 매핑
- 각 inspection_id별로 21장의 이미지가 모두 있는지 확인 (일부 inspection_id는 21장이 안 될 수도 있음)

1. 기본 데이터 구조: 총 이미지 수, 어노테이션 수, 카테고리 정보
2. Inspection ID 분석: 각 inspection_id별로 몇 장의 이미지가 있는지
3. 이미지 개수 분포: 21장을 가진 inspection_id가 몇 개인지, 그렇지 않은 것은 몇 개인지
4. 카메라-트리거 조합: 실제로 어떤 (cam_name, trigger_mm) 조합들이 존재하는지
5. 어노테이션 분석: 각 구멍 카테고리별로 얼마나 많은 어노테이션이 있는지, 마스킹된 것은 몇 개인지

In [ ]:
import json
import pandas as pd
from collections import defaultdict, Counter

# 1. JSON 파일 로드
file_path = "/content/real_labeled_test_set_masked_images/coco_segmentations.json"

with open(file_path, 'r') as f:
    coco_data = json.load(f)

print("=== COCO 데이터셋 기본 정보 ===")
print(f"총 이미지 수: {len(coco_data['images'])}")
print(f"총 어노테이션 수: {len(coco_data['annotations'])}")
print(f"카테고리 수: {len(coco_data['categories'])}")

# 2. 카테고리 정보 확인
print("\n=== 카테고리 정보 ===")
for cat in coco_data['categories']:
    print(f"ID: {cat['id']}, Name: {cat['name']}")

# 3. inspection_id별 이미지 개수 분석
inspection_images = defaultdict(list)
cam_trigger_combinations = set()

for img in coco_data['images']:
    inspection_id = img['inspection_id']
    cam_name = img['cam_name']
    trigger_mm = img['trigger_mm']

    inspection_images[inspection_id].append({
        'image_id': img['id'],
        'file_name': img['file_name'],
        'cam_name': cam_name,
        'trigger_mm': trigger_mm
    })

    cam_trigger_combinations.add((cam_name, trigger_mm))

print(f"\n=== Inspection ID 분석 ===")
print(f"총 inspection_id 개수: {len(inspection_images)}")
print(f"inspection_id 범위: {min(inspection_images.keys())} ~ {max(inspection_images.keys())}")

# 4. 각 inspection_id별 이미지 개수 분포
image_counts = [len(images) for images in inspection_images.values()]
image_count_distribution = Counter(image_counts)

print(f"\n=== Inspection ID별 이미지 개수 분포 ===")
for count, frequency in sorted(image_count_distribution.items()):
    print(f"{count}장: {frequency}개 inspection_id")

# 5. 카메라-트리거 조합 분석
print(f"\n=== 카메라-트리거 조합 분석 ===")
print(f"총 (cam_name, trigger_mm) 조합 수: {len(cam_trigger_combinations)}")
print("가능한 조합들:")
for cam, trigger in sorted(cam_trigger_combinations):
    print(f"  {cam} - {trigger}mm")

# 6. 상세 분석: 각 inspection_id가 어떤 조합을 가지고 있는지
print(f"\n=== 각 Inspection ID별 상세 정보 (처음 5개) ===")
for inspection_id in sorted(list(inspection_images.keys()))[:5]:
    images = inspection_images[inspection_id]
    print(f"\nInspection ID {inspection_id}: {len(images)}장")
    for img in images:
        print(f"  - {img['file_name']}: {img['cam_name']}, {img['trigger_mm']}mm")

# 7. 21장이 아닌 inspection_id 찾기
print(f"\n=== 21장이 아닌 Inspection ID들 ===")
non_21_inspections = []
for inspection_id, images in inspection_images.items():
    if len(images) != 21:
        non_21_inspections.append((inspection_id, len(images)))

if non_21_inspections:
    print("21장이 아닌 inspection_id:")
    for inspection_id, count in sorted(non_21_inspections):
        print(f"  Inspection ID {inspection_id}: {count}장")
else:
    print("모든 inspection_id가 21장씩 가지고 있습니다.")

# 8. 어노테이션 분석 (구멍 탐지 정보)
print(f"\n=== 어노테이션 분석 ===")
category_counts = Counter()
masked_counts = Counter()

for ann in coco_data['annotations']:
    category_counts[ann['category_id']] += 1
    if ann.get('is_masked', 0) == 1:
        masked_counts[ann['category_id']] += 1

print("카테고리별 어노테이션 개수:")
for cat in coco_data['categories']:
    cat_id = cat['id']
    cat_name = cat['name']
    total = category_counts[cat_id]
    masked = masked_counts[cat_id]
    print(f"  {cat_name} (ID:{cat_id}): 총 {total}개, 마스킹된 것 {masked}개")

# 9. 데이터 구조를 DataFrame으로 정리
inspection_summary = []
for inspection_id, images in inspection_images.items():
    inspection_summary.append({
        'inspection_id': inspection_id,
        'image_count': len(images),
        'cam_trigger_combinations': len(set((img['cam_name'], img['trigger_mm']) for img in images))
    })

df_summary = pd.DataFrame(inspection_summary)
print(f"\n=== 요약 통계 ===")
print(f"평균 이미지 수: {df_summary['image_count'].mean():.2f}")
print(f"최소 이미지 수: {df_summary['image_count'].min()}")
print(f"최대 이미지 수: {df_summary['image_count'].max()}")
print(f"평균 카메라-트리거 조합 수: {df_summary['cam_trigger_combinations'].mean():.2f}")

=== COCO 데이터셋 기본 정보 ===
총 이미지 수: 1680
총 어노테이션 수: 6697
카테고리 수: 7

=== 카테고리 정보 ===
ID: 0, Name: AX1
ID: 1, Name: BY1
ID: 2, Name: CY1
ID: 3, Name: DY1
ID: 4, Name: DY2
ID: 5, Name: DY3
ID: 6, Name: DY4

=== Inspection ID 분석 ===
총 inspection_id 개수: 86
inspection_id 범위: 1 ~ 86

=== Inspection ID별 이미지 개수 분포 ===
3장: 1개 inspection_id
4장: 1개 inspection_id
6장: 2개 inspection_id
9장: 2개 inspection_id
12장: 2개 inspection_id
15장: 2개 inspection_id
17장: 1개 inspection_id
18장: 1개 inspection_id
21장: 74개 inspection_id

=== 카메라-트리거 조합 분석 ===
총 (cam_name, trigger_mm) 조합 수: 21
가능한 조합들:
  Z1CAM02 - 700mm
  Z1CAM02 - 1200mm
  Z1CAM02 - 1700mm
  Z1CAM03 - 1200mm
  Z1CAM04 - 1700mm
  Z1CAM04 - 1800mm
  Z1CAM05 - 700mm
  Z1CAM05 - 1100mm
  Z1CAM05 - 1200mm
  Z1CAM05 - 1300mm
  Z1CAM06 - 1200mm
  Z1CAM06 - 1300mm
  Z1CAM06 - 1700mm
  Z1CAM07 - 700mm
  Z1CAM07 - 800mm
  Z1CAM07 - 1100mm
  Z1CAM07 - 1200mm
  Z1CAM08 - 1200mm
  Z1CAM08 - 1300mm
  Z1CAM08 - 1500mm
  Z1CAM08 - 1700mm

=== 각 Inspection ID별 상세 정보 (처음 5개) 

### 📌 완전한 데이터(21장의 이미지)를 가진 inspection_id
- 74개의 inspection_id가 21장씩 완전한 데이터 보유
- 12개의 inspection_id가 3~18장으로 불완전한 데이터 (12, 13, 27, 41, 65, 80, 81, 82, 83, 84, 85, 86)

### 📌어노테이션 분포
- 총 6,697개의 구멍 어노테이션
- BY1, DY1, DY2가 가장 많이 탐지됨 (1,000개 이상)
- AX1이 가장 적게 탐지됨 (354개)
- 전체 어노테이션의 약 10.5%가 마스킹됨 (결함 시뮬레이션용)

# 1-2단계: 추가 분석
1️⃣ 각 inspection_id별 7개 카테고리 존재 여부
- 마스킹되지 않은 어노테이션만 "존재하는" 카테고리로 간주
- 완전한 데이터(21장)를 가진 inspection_id들에 대해서만 분석
- 정상품 vs 결함품 구분

2️⃣ 21장 미만 inspection_id 처리 방침
동의합니다! 21장 미만의 inspection_id들은 분석에서 제외하는 것이 맞습니다. 이유:

- 모든 각도/거리에서 촬영되지 않아 구멍 탐지가 불완전할 수 있음
- 실제 생산 환경에서도 모든 카메라-트리거 조합에서 촬영이 필요
- 품질 관리의 일관성을 위해 완전한 데이터만 사용해야 함

3️⃣ 마스킹 분포 분석

- 어떤 inspection_id에서 어떤 카테고리가 마스킹되었는지 상세 분석
- 마스킹 = 결함 시뮬레이션 (구멍이 뚫리지 않은 상황)
- 카테고리별 마스킹 빈도 통계

In [ ]:
import json
import pandas as pd
from collections import defaultdict, Counter

# JSON 파일 로드
file_path = "/content/real_labeled_test_set_masked_images/coco_segmentations.json"

with open(file_path, 'r') as f:
    coco_data = json.load(f)

# image_id와 inspection_id 매핑 생성
image_to_inspection = {}
for img in coco_data['images']:
    image_to_inspection[img['id']] = img['inspection_id']

# 카테고리 이름 매핑
category_names = {cat['id']: cat['name'] for cat in coco_data['categories']}
all_categories = set(category_names.keys())  # {0, 1, 2, 3, 4, 5, 6}

print("=== 1. 각 inspection_id별로 7개 카테고리 존재 여부 확인 ===")

# inspection_id별 카테고리 존재 여부 분석
inspection_categories = defaultdict(set)
inspection_masked_categories = defaultdict(set)

for ann in coco_data['annotations']:
    image_id = ann['image_id']
    inspection_id = image_to_inspection[image_id]
    category_id = ann['category_id']
    is_masked = ann.get('is_masked', 0)

    # 마스킹되지 않은 경우만 "존재하는" 카테고리로 간주
    if is_masked == 0:
        inspection_categories[inspection_id].add(category_id)
    else:
        inspection_masked_categories[inspection_id].add(category_id)

# 완전한 데이터를 가진 inspection_id들 (21장)
complete_inspections = []
incomplete_inspections = []

inspection_images = defaultdict(list)
for img in coco_data['images']:
    inspection_images[img['inspection_id']].append(img)

for inspection_id, images in inspection_images.items():
    if len(images) == 21:
        complete_inspections.append(inspection_id)
    else:
        incomplete_inspections.append(inspection_id)

print(f"완전한 데이터 (21장): {len(complete_inspections)}개")
print(f"불완전한 데이터 (21장 미만): {len(incomplete_inspections)}개")

# 완전한 데이터에서 카테고리 분석
print(f"\n=== 완전한 데이터 (21장) inspection_id들의 카테고리 분석 ===")

perfect_inspections = []  # 7개 카테고리 모두 존재
defective_inspections = []  # 일부 카테고리 누락 (결함품)

category_analysis = []

for inspection_id in sorted(complete_inspections):
    existing_categories = inspection_categories[inspection_id]
    masked_categories = inspection_masked_categories[inspection_id]
    missing_categories = all_categories - existing_categories

    category_analysis.append({
        'inspection_id': inspection_id,
        'existing_count': len(existing_categories),
        'missing_count': len(missing_categories),
        'masked_count': len(masked_categories),
        'existing_categories': sorted(list(existing_categories)),
        'missing_categories': sorted(list(missing_categories)),
        'masked_categories': sorted(list(masked_categories))
    })

    if len(existing_categories) == 7:
        perfect_inspections.append(inspection_id)
    else:
        defective_inspections.append(inspection_id)

print(f"완벽한 제품 (7개 카테고리 모두 존재): {len(perfect_inspections)}개")
print(f"결함 제품 (일부 카테고리 누락): {len(defective_inspections)}개")

# 상세 결과 출력
print(f"\n=== 결함 제품 상세 분석 ===")
for analysis in category_analysis:
    if analysis['missing_count'] > 0:
        inspection_id = analysis['inspection_id']
        existing = [category_names[cat_id] for cat_id in analysis['existing_categories']]
        missing = [category_names[cat_id] for cat_id in analysis['missing_categories']]
        masked = [category_names[cat_id] for cat_id in analysis['masked_categories']]

        print(f"\nInspection ID {inspection_id}:")
        print(f"  존재하는 구멍 ({len(existing)}개): {existing}")
        print(f"  누락된 구멍 ({len(missing)}개): {missing}")
        if masked:
            print(f"  마스킹된 구멍 ({len(masked)}개): {masked}")

print(f"\n=== 3. 마스킹된 어노테이션 분포 분석 ===")

# 마스킹 통계
masking_stats = defaultdict(lambda: defaultdict(int))
total_masking_by_inspection = defaultdict(int)

for ann in coco_data['annotations']:
    if ann.get('is_masked', 0) == 1:
        image_id = ann['image_id']
        inspection_id = image_to_inspection[image_id]
        category_id = ann['category_id']

        masking_stats[inspection_id][category_id] += 1
        total_masking_by_inspection[inspection_id] += 1

print(f"마스킹이 적용된 inspection_id 개수: {len(masking_stats)}")

# 마스킹된 inspection_id들과 카테고리별 분포
print(f"\n=== 마스킹된 inspection_id별 상세 분석 ===")
for inspection_id in sorted(masking_stats.keys()):
    if inspection_id in complete_inspections:  # 완전한 데이터만 분석
        print(f"\nInspection ID {inspection_id} (총 마스킹 수: {total_masking_by_inspection[inspection_id]}):")
        for category_id, count in sorted(masking_stats[inspection_id].items()):
            category_name = category_names[category_id]
            print(f"  {category_name}: {count}개 마스킹")

# 마스킹 통계 요약
print(f"\n=== 마스킹 통계 요약 ===")
masked_inspection_ids = [iid for iid in masking_stats.keys() if iid in complete_inspections]
print(f"완전한 데이터 중 마스킹이 적용된 inspection_id: {len(masked_inspection_ids)}개")
print(f"마스킹이 적용되지 않은 inspection_id: {len(complete_inspections) - len(masked_inspection_ids)}개")

# 카테고리별 마스킹 빈도
category_masking_freq = Counter()
for inspection_id, categories in masking_stats.items():
    if inspection_id in complete_inspections:
        for category_id in categories.keys():
            category_masking_freq[category_id] += 1

print(f"\n카테고리별 마스킹 적용된 inspection_id 수:")
for category_id in sorted(category_masking_freq.keys()):
    category_name = category_names[category_id]
    freq = category_masking_freq[category_id]
    print(f"  {category_name}: {freq}개 inspection_id에서 마스킹")

# 최종 요약
print(f"\n=== 최종 요약 ===")
print(f"전체 inspection_id: {len(inspection_images)}개")
print(f"  - 완전한 데이터 (21장): {len(complete_inspections)}개")
print(f"    - 정상품 (7개 구멍 모두 존재): {len(perfect_inspections)}개")
print(f"    - 결함품 (일부 구멍 누락): {len(defective_inspections)}개")
print(f"  - 불완전한 데이터 (21장 미만): {len(incomplete_inspections)}개 → 분석 제외 권장")

# DataFrame으로 정리하여 CSV 저장 가능하도록 준비
df_analysis = pd.DataFrame(category_analysis)
print(f"\n완전한 데이터 inspection_id들의 통계:")
print(f"존재하는 카테고리 수 분포:")
print(df_analysis['existing_count'].value_counts().sort_index())

=== 1. 각 inspection_id별로 7개 카테고리 존재 여부 확인 ===
완전한 데이터 (21장): 74개
불완전한 데이터 (21장 미만): 12개

=== 완전한 데이터 (21장) inspection_id들의 카테고리 분석 ===
완벽한 제품 (7개 카테고리 모두 존재): 74개
결함 제품 (일부 카테고리 누락): 0개

=== 결함 제품 상세 분석 ===

=== 3. 마스킹된 어노테이션 분포 분석 ===
마스킹이 적용된 inspection_id 개수: 79

=== 마스킹된 inspection_id별 상세 분석 ===

Inspection ID 1 (총 마스킹 수: 11):
  BY1: 4개 마스킹
  DY1: 1개 마스킹
  DY2: 5개 마스킹
  DY3: 1개 마스킹

Inspection ID 2 (총 마스킹 수: 6):
  AX1: 1개 마스킹
  BY1: 3개 마스킹
  DY3: 1개 마스킹
  DY4: 1개 마스킹

Inspection ID 3 (총 마스킹 수: 11):
  AX1: 1개 마스킹
  BY1: 1개 마스킹
  DY1: 3개 마스킹
  DY2: 5개 마스킹
  DY4: 1개 마스킹

Inspection ID 4 (총 마스킹 수: 9):
  BY1: 3개 마스킹
  CY1: 1개 마스킹
  DY1: 2개 마스킹
  DY3: 3개 마스킹

Inspection ID 5 (총 마스킹 수: 5):
  BY1: 2개 마스킹
  DY2: 3개 마스킹

Inspection ID 6 (총 마스킹 수: 9):
  BY1: 2개 마스킹
  DY1: 3개 마스킹
  DY2: 2개 마스킹
  DY4: 2개 마스킹

Inspection ID 7 (총 마스킹 수: 10):
  AX1: 1개 마스킹
  BY1: 2개 마스킹
  CY1: 1개 마스킹
  DY2: 2개 마스킹
  DY3: 2개 마스킹
  DY4: 2개 마스킹

Inspection ID 8 (총 마스킹 수: 7):
  AX1: 3개 마스킹
  CY1: 1개 마스킹
  DY3: 1개 마스킹


### 📌 real_labeled_test_set_masked_images 데이터의 특성
- 모든 완전한 데이터(74개 inspection_id)에서 7개 카테고리가 모두 존재
- 하지만 모든 inspection_id에서 마스킹이 적용됨 (74개 모두)
- 즉, 마스킹 = 결함 시뮬레이션이 모든 제품에 적용된 테스트 데이터셋
- 마스킹된 어노테이션들이 바로 YOLO 모델이 찾지 못해야 하는 결함

### 📌 마스킹 패턴 분석
- BY1이 가장 자주 마스킹됨 (62개 inspection_id)
- AX1이 가장 적게 마스킹됨 (23개 inspection_id)
- 각 inspection_id당 평균 약 8-9개의 어노테이션이 마스킹